# **Bitcoin Buy, Hold, Sell decision making project**
---

In [1]:
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Lambda
from keras import optimizers
from keras.utils import np_utils

import numpy as np
import math

Using TensorFlow backend.


**X : 시가,고가, 저가, 종가, 누적거래금액, 누적거래량**

In [2]:
rawData = np.loadtxt("priceData.csv", delimiter=',')

In [3]:
X = rawData[:, 1:]
Y = np.zeros((len(X), 3))

### Normalization and Standardization

In [4]:
def normalize(data):
    return (data - np.min(data, axis = 0))/(np.max(data, axis = 0) - np.min(data, axis = 0))

In [5]:
def standardize(data) :
    return (data - np.mean(data, axis = 0))/np.std(data, axis = 0)

In [6]:
X = normalize(X)
X = standardize(X)

## Functions
---
make_BuyHoldSell_point()<br/>print_BuyHoldSell_point()

### make_BuyHoldSell_point
This function makes Buy, Hold, Sell point. (Y)
np.ndarray Y.shape is (len(X), 3)
- Buy : Y[ , 0] = 1
- Hold : Y[ , 1] = 1
- Sell : Y[ , 2] = 1

In [7]:
period = 360 # 60분 * 24시간 * 3일
period = (int)(period/2)

In [8]:
def make_BuyHoldSell_point(X, Y) :
    
    for i in range(period, len(X)-period) :

        arr = X[i-period:i+period, 3]

        minIndex = np.argmin(arr)
        maxIndex = np.argmax(arr)

        Y[i-period:i+period, :] = 0
        Y[i-period + minIndex, 0] = 1
        Y[i-period + maxIndex, 2] = 1
        
    for i in range(0, len(X)) :
        if Y[i, 0] == 0 and Y[i, 2] == 0 :
            Y[i, 1] = 1

### print_BuyHoldSell_point
This function shows time, BuyHoldSell, price.

In [9]:
def print_BuyHoldSell_point(X, Y) :
    for i in range(0, len(X)) :
        if Y[i, 0] == 1 :
            print(i, "\tBuy\t", X[i, 3])
        elif Y[i, 2] == 1 :
            print(i, "\tSell\t", X[i, 3])
        else :
            print(i, "\tHold\t", X[i, 3])

## Building an RNN with LSTM
---

In [10]:
make_BuyHoldSell_point(X, Y)
#print_BuyHoldSell_point(X, Y)

In [48]:
X_train, X_test = X[:910600], X[910600:]
Y_train, Y_test = Y[:910600], Y[910600:]

#X_train = X_train.reshape((1, X_train.shape[0], X_train.shape[1]))
#X_test = X_test.reshape((1, X_test.shape[0], X_test.shape[1]))
#Y_train = Y_train.reshape((1, Y_train.shape[0], Y_train.shape[1]))
#Y_test = Y_test.reshape((1, Y_test.shape[0], Y_test.shape[1]))

In [49]:
print(X_train.shape)
print(Y_train.shape)

(910600, 6)
(910600, 3)


### Setting the RNN Parameters

In [31]:
batch_size = 1            # Batch size (you may try different values)
batch_length = (int)(910600/100)  # 9106
epochs = 15               # Epoch (you may try different values)
seq_len = 60 * 24 * 3     # 4320 sequence data (Representing the last 3 days)
loss='mean_squared_error' # Since the metric is MSE/RMSE
opt = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01) #imizer = 'rmsprop'     # Recommended optimizer for RNN
activation = 'relu'       # Linear activation
input_shape=(None,6)      # Input dimension
output_dim = 32           # Output dimension

### Creating a Sequential Model

In [14]:
model = Sequential()
model.add(LSTM(units=output_dim, return_sequences=True, input_shape=input_shape))
model.add(Dense(units=output_dim,activation=activation))
model.add(LSTM(units=output_dim, return_sequences=False))
model.add(Dense(units=3,activation='softmax'))
model.compile(optimizer=opt,loss='categorical_crossentropy')

### Training the RNN

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, None, 32)          4992      
_________________________________________________________________
dense_1 (Dense)              (None, None, 32)          1056      
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 32)          8320      
_________________________________________________________________
dense_2 (Dense)              (None, None, 3)           99        
Total params: 14,467
Trainable params: 14,467
Non-trainable params: 0
_________________________________________________________________


In [62]:
def trainning_Model(X_train, Y_train, batch_length) :
    
    for batch in range(math.ceil(len(X_train)/batch_length)) :
    
        inputData = np.zeros((0, Tx, 6))    # (m, Tx, 6)
        outputData = np.zeros((0, 1, 3))    # (m, Tx, 6)
    
        for i in range(batch_length) :

            if batch*batch_length + i + Tx < len(X_train) :
                X_input = X_train[batch*batch_length + i : batch*batch_length + i + Tx, :]
            else :
                X_input = X_train[batch*batch_length + i:, :]

            X_input = X_input.reshape((1, X_input.shape[0], X_input.shape[1]))
            inputData = np.append(inputData, X_input, axis = 0)
            outputData = np.append(outputData, Y_train[batch*batch_length + i, :].reshape((1, 1, 3)), axis = 0)
        
        print(inputData.shape, outputData.shape)
        model.fit(x=inputData, y=outputData, epochs=epochs, validation_split=0.00)
        
    model.save('predict_model.h5')

In [63]:
trainning_Model(X_train, Y_train, batch_length)

(9106, 10, 6) (9106, 1, 3)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/15


KeyboardInterrupt: 

### Notebook

In [ ]:
print_BuyHoldSell_point(X, Y)